In [2]:
import requests, argparse, warnings, mmcv, torch
from mmcv import Config, DictAction
from mmcv.cnn import fuse_conv_bn
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
from mmdet.datasets import build_dataloader, build_dataset
from mmdet.models import build_detector
import mmdet_custom  # noqa: F401,F403
import mmcv_custom  # noqa: F401,F403

/workspace/fisheye_challenge/ops_dcnv3/modules/dcnv3.py:20: UserWarning: Now, we support DCNv4 in InternImage.
  warnings.warn('Now, we support DCNv4 in InternImage.')


In [3]:
config = "configs//cascade_internimage_xl_fpn_3x_coco.py"
cfg = Config.fromfile(config)

In [4]:
torch.backends.cudnn.benchmark = True
cfg.model.pretrained = None
cfg.data.test.test_mode = True

In [5]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=4, dist=False, shuffle=False)
data_iterator = iter(data_loader)
data = next(data_iterator)
data

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


{'img_metas': [DataContainer([[{'filename': '/workspace/FishEye8k/dataset/Fisheye8K_all_including_train/test/images/camera2_A_167.png', 'ori_filename': 'camera2_A_167.png', 'ori_shape': (1088, 1085, 3), 'img_shape': (802, 800, 3), 'pad_shape': (832, 800, 3), 'scale_factor': array([0.7373272, 0.7371324, 0.7373272, 0.7371324], dtype=float32), 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}]])],
 'img': [DataContainer([tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
            [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
            [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
            ...,
            [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
            [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
            [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  

In [6]:
data["img"][0]

DataContainer([tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8

In [7]:
checkpoint = "checkpoints/cascade_internimage_xl_fpn_3x_coco.pth"
cfg.model.train_cfg = None
model = build_detector(cfg.model) # cfg.model contains the threshold for nms as well as score threshold
checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')
model = fuse_conv_bn(model)
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model, device_ids=[0])

2024-03-29 22:50:20,004 - mmdet - INFO - using core type: DCNv3
2024-03-29 22:50:20,005 - mmdet - INFO - using activation layer: GELU
2024-03-29 22:50:20,005 - mmdet - INFO - using main norm layer: LN
2024-03-29 22:50:20,005 - mmdet - INFO - using dpr: linear, 0.6
2024-03-29 22:50:20,006 - mmdet - INFO - level2_post_norm: False
2024-03-29 22:50:20,006 - mmdet - INFO - level2_post_norm_block_ids: None
2024-03-29 22:50:20,006 - mmdet - INFO - res_post_norm: False
2024-03-29 22:50:20,007 - mmdet - INFO - use_dcn_v4_op: False


load checkpoint from local path: checkpoints/cascade_internimage_xl_fpn_3x_coco.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: roi_head.mask_head.0.convs.0.conv.weight, roi_head.mask_head.0.convs.0.conv.bias, roi_head.mask_head.0.convs.1.conv.weight, roi_head.mask_head.0.convs.1.conv.bias, roi_head.mask_head.0.convs.2.conv.weight, roi_head.mask_head.0.convs.2.conv.bias, roi_head.mask_head.0.convs.3.conv.weight, roi_head.mask_head.0.convs.3.conv.bias, roi_head.mask_head.0.upsample.weight, roi_head.mask_head.0.upsample.bias, roi_head.mask_head.0.conv_logits.weight, roi_head.mask_head.0.conv_logits.bias, roi_head.mask_head.1.convs.0.conv.weight, roi_head.mask_head.1.convs.0.conv.bias, roi_head.mask_head.1.convs.1.conv.weight, roi_head.mask_head.1.convs.1.conv.bias, roi_head.mask_head.1.convs.2.conv.weight, roi_head.mask_head.1.convs.2.conv.bias, roi_head.mask_head.1.convs.3.conv.weight, roi_head.mask_head.1.convs.3.conv.bias, roi_head.mask_h

In [8]:
model

MMDataParallel(
  (module): CascadeRCNN(
    (backbone): InternImage(
      (patch_embed): StemLayer(
        (conv1): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm1): Sequential(
          (0): to_channels_last()
          (1): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (2): to_channels_first()
        )
        (act): GELU()
        (conv2): Conv2d(96, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm2): Sequential(
          (0): to_channels_last()
          (1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
        )
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (levels): ModuleList(
        (0): InternImageBlock(
          (blocks): ModuleList(
            (0): InternImageLayer(
              (norm1): Sequential(
                (0): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
              )
              (dcn): DCNv3(
                (dw_conv): Sequential(
                 

In [ ]:
result = model(return_loss=False, rescale=True, **data)